In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from datetime import date

In [ ]:
# Get reviews from a single page
def get_reviews(appid, params={'json':1}):
  # Convert App-ID to string
  appid = str(appid)
  url = "https://store.steampowered.com/appreviews/"
  # Get the page source
  response = requests.get(url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
  # Return as json format
  return response.json()

# Get a batch of reviews using cursor
def get_batch_reviews(appid, sentiment, filter, n, use_date, date_input='0', language='english'):

  date_input = time.mktime(datetime.strptime(date_input, "%d-%m-%Y").timetuple())

  appid = str(appid)
  reviews = []
  cursor = '*'

  # Parameters for the API
  params = {'json':1,
            'filter':filter,
            'language':language,
            'day_range': 365,
            'cursor': cursor.encode(),
            'review_type':sentiment}

  if use_date:
    params['filter']='recent'
    # Today's date
    oldest_post = time.mktime(datetime.strptime(date.today().strftime("%d-%m-%Y"), "%d-%m-%Y").timetuple())
    while True:
      newest_post = oldest_post
      # Number of reviews that should be obtained in every page
      params['num_per_page'] = min(n, 100)
      # Grab the reviews for a single page
      response = get_reviews(appid, params)
      # Pass the new cursor to scrape the next page/batch
      params['cursor'] = response['cursor']
      # Save to a list
      reviews += response['reviews']

      # Update the oldest post
      oldest_post = reviews[-1]['timestamp_created']

      # Check the timestamp of the last review (oldest by date)
      # print(reviews[-1]['timestamp_created'])
      if (reviews[-1]['timestamp_created']<date_input) or (newest_post==oldest_post):
        break

  else:
    # Loop through the "pages" until we get reviews at least as many as we want (may get more!)
    while n > len(reviews):
      # Number of reviews that should be obtained in every page
      params['num_per_page'] = min(n, 100)

      # Grab the reviews for a single page
      response = get_reviews(appid, params)

      # Pass the new cursor to scrape the next page/batch
      params['cursor'] = response['cursor']

      # Save to a list
      reviews += response['reviews']

      # Get n reviews (sometimes the API does not return 100 reviews per page, 
      # resulting in number of reviews gathered > n)
      reviews = reviews[0:n]

    # Debugging tool
    # print(f'Reviews added = {len(reviews)}')

  return reviews

In [ ]:
# Load the game list
games = pd.read_csv('game_list_new.csv')

# Features to be extracted from the API
features = ['author', 'review', 'timestamp_created', 'voted_up', 'votes_up', 'votes_funny']

In [ ]:
# Get recent reviews since a specified date up until now for each game
# DataFrame to store the results
df_recent = pd.DataFrame()

# Get reviews since this date
date_input="01-01-2022"

for i in range(len(games['id'])):
  game_name = games.loc[i, 'name']
  print(f'Fetching reviews of {game_name}')
  reviews = pd.DataFrame(get_batch_reviews(str(games.loc[i]['id']), 'all', 'recent', 100, True, date_input=date_input))[features]
  reviews['name'] = game_name
  df_recent = pd.concat([df_recent, reviews], axis=0)
  print('Finished. \n')

# Sort by date and save the file
df_recent[df_recent['timestamp_created']>time.mktime(datetime.strptime(date_input, "%d-%m-%Y").timetuple())].sort_values(by='timestamp_created').to_csv('recent_reviews_2022.csv', index=False)

Fetching reviews of Cyberpunk 2077
Finished. 

Fetching reviews of DayZ
Finished. 

Fetching reviews of New World
Finished. 

Fetching reviews of No Man's Sky
Finished. 

Fetching reviews of Battlefield™ 2042
Finished. 

Fetching reviews of NARAKA: BLADEPOINT
Finished. 

Fetching reviews of Battlefield V
Finished. 

Fetching reviews of Grand Theft Auto IV: The Complete Edition
Finished. 

Fetching reviews of Dying Light 2 Stay Human
Finished. 

Fetching reviews of Elite Dangerous
Finished. 

Fetching reviews of SCUM
Finished. 

Fetching reviews of Tom Clancy's Ghost Recon® Wildlands
Finished. 

Fetching reviews of Conan Exiles
Finished. 

Fetching reviews of Microsoft Flight Simulator Game of the Year Edition
Finished. 

Fetching reviews of Total War: WARHAMMER III
Finished. 

Fetching reviews of Fallout 76: The Pitt
Finished. 

Fetching reviews of Resident Evil 3
Finished. 

Fetching reviews of Stranded Deep
Finished. 

Fetching reviews of UNO
Finished. 

Fetching reviews of Back 4 Bl

In [ ]:
# Get 1000 most helpful reviews from each game
# DataFrame to store the results
df_helpful = pd.DataFrame()
for i in range(len(games['id'])):
  game_name = games.loc[i, 'name']
  print(f'Fetching reviews of {game_name}')
  reviews = pd.DataFrame(get_batch_reviews(str(games.loc[i]['id']), 'all', 'all', 1000, False))[features]
  reviews['name'] = game_name
  df_helpful = pd.concat([df_helpful, reviews], axis=0)
  print('Finished. \n')

# Shuffle the dataframe and save the file
df_helpful.sample(frac=1, random_state=42).reset_index(drop=True).to_csv('helpful_reviews.csv', index=False)

Fetching reviews of Cyberpunk 2077
Finished. 

Fetching reviews of DayZ
Finished. 

Fetching reviews of New World
Finished. 

Fetching reviews of No Man's Sky
Finished. 

Fetching reviews of Battlefield™ 2042
Finished. 

Fetching reviews of NARAKA: BLADEPOINT
Finished. 

Fetching reviews of Battlefield V
Finished. 

Fetching reviews of Grand Theft Auto IV: The Complete Edition
Finished. 

Fetching reviews of Dying Light 2 Stay Human
Finished. 

Fetching reviews of Elite Dangerous
Finished. 

Fetching reviews of SCUM
Finished. 

Fetching reviews of Tom Clancy's Ghost Recon® Wildlands
Finished. 

Fetching reviews of Conan Exiles
Finished. 

Fetching reviews of Microsoft Flight Simulator Game of the Year Edition
Finished. 

Fetching reviews of Total War: WARHAMMER III
Finished. 

Fetching reviews of Fallout 76: The Pitt
Finished. 

Fetching reviews of Resident Evil 3
Finished. 

Fetching reviews of Stranded Deep
Finished. 

Fetching reviews of UNO
Finished. 

Fetching reviews of Back 4 Bl

In [ ]:
# Get 1000 balanced reviews (500 good, 500 bad) from each game (based on helpfulness)
# DataFrame to store the results
df = pd.DataFrame()
for i in range(len(games['id'])):
  
  game_name = games.loc[i, 'name'] 

  print(f'Fetching reviews of {game_name}')
  # Get 500 (or more) good reviews
  reviews = pd.DataFrame(get_batch_reviews(str(games.loc[i, 'id']), 'positive', 'all', 500, False))[features]
  reviews['name'] = game_name
  df = pd.concat([df, reviews], axis=0)

  # Get 500 (or more) bad reviews
  reviews = pd.DataFrame(get_batch_reviews(str(games.loc[i, 'id']), 'negative', 'all', 500, False))[features]
  reviews['name'] = game_name
  df = pd.concat([df, reviews], axis=0)
  print('Finished. \n')

# Shuffle the dataframe and save the file
df.sample(frac=1, random_state=42).reset_index(drop=True).to_csv('balanced_reviews.csv', index=False)

Fetching reviews of Cyberpunk 2077
Finished. 

Fetching reviews of DayZ
Finished. 

Fetching reviews of New World
Finished. 

Fetching reviews of No Man's Sky
Finished. 

Fetching reviews of Battlefield™ 2042
Finished. 

Fetching reviews of NARAKA: BLADEPOINT
Finished. 

Fetching reviews of Battlefield V
Finished. 

Fetching reviews of Grand Theft Auto IV: The Complete Edition
Finished. 

Fetching reviews of Dying Light 2 Stay Human
Finished. 

Fetching reviews of Elite Dangerous
Finished. 

Fetching reviews of SCUM
Finished. 

Fetching reviews of Tom Clancy's Ghost Recon® Wildlands
Finished. 

Fetching reviews of Conan Exiles
Finished. 

Fetching reviews of Microsoft Flight Simulator Game of the Year Edition
Finished. 

Fetching reviews of Total War: WARHAMMER III
Finished. 

Fetching reviews of Fallout 76: The Pitt
Finished. 

Fetching reviews of Resident Evil 3
Finished. 

Fetching reviews of Stranded Deep
Finished. 

Fetching reviews of UNO
Finished. 

Fetching reviews of Back 4 Bl